## Preliminary preparations

In [1]:
from datetime import datetime
import os
from pathlib import Path

import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
pd.set_option('display.max_rows', 100)

In [2]:
logfile = "../logs/128G/transaction-only-128GB-2024-10-07-06-55-38-mainnet-beta.log"
csv = "../logs/128G/tps_128gb.csv"

## Preparing Data Source

In [3]:
data = []

with open(logfile, 'r') as file:
    for line in file:
        if "datapoint: replay-slot-stats" in line:

            timestamp_match = re.search(r"\[(.*?)Z", line)
            transactions_match = re.search(r"total_transactions=(\d+)i", line)
            execute_us_match = re.search(r"execute_us=(\d+)i", line)
            
            if timestamp_match and transactions_match and execute_us_match:
                timestamp = timestamp_match.group(1)
                total_transactions_executed = int(transactions_match.group(1))
                execute_us = int(execute_us_match.group(1))

                data.append([timestamp, total_transactions_executed, execute_us])

df = pd.DataFrame(data, columns=['timestamp', 'total_transactions_executed', 'execute_us'])
df

FileNotFoundError: [Errno 2] No such file or directory: '../logs/128G/transaction-only-128GB-2024-10-07-06-55-38-mainnet-beta.log'

In [ ]:
def convert_to_unixtime(timestamp):
    base_time, microseconds = timestamp.split('.')
    microseconds = microseconds[:6]
    dt = datetime.strptime(base_time, '%Y-%m-%dT%H:%M:%S')
    return int(dt.timestamp() * 1_000_000) + int(microseconds)

In [ ]:
df['end'] = df['timestamp'].apply(convert_to_unixtime)
df['start'] = df['end'] - df['execute_us']
df['tps'] = df['total_transactions_executed'] * 1000000 / df['execute_us']
df

In [ ]:
start_time = df.start.min() // 1000000 * 1000000
end_time = (df.start.max() + 1000000) // 1000000 * 1000000
time_step = 100000 # 10,000 microsends = .1 second

time_series = pd.DataFrame({
    't': np.arange(start_time, end_time + time_step, time_step)
})

def aggregate_tps(row, df):
    mask = (df['start'] <= row['t']) & (df['end'] >= row['t'])
    return df.loc[mask, 'tps'].sum()

time_series['sum_tps'] = time_series.apply(aggregate_tps, axis=1, df=df)
# window=10 means creating a moving average with the past 1 second's data. If you change it to 100, it would be the past 10 seconds
time_series['moving_average'] = time_series['sum_tps'].rolling(window=50, min_periods=1).mean()
time_series.to_csv(csv, index=False)


In [ ]:
# Calculate the maximum TPS
max_tps = time_series['sum_tps'].max()

# Calculate the overall average TPS
average_tps = time_series['sum_tps'].mean()

# Print the results
print(f"Maximum TPS: {max_tps:,.2f}")
print(f"Average TPS: {average_tps:,.2f}")

# Convert the max TPS x-coordinate to the corresponding time
max_tps_time = time_series.loc[time_series['sum_tps'].idxmax(), 't']

# Calculate the total duration spent with 0 TPS
time_with_zero_tps = time_series[time_series['sum_tps'] == 0]
total_time_zero_tps = len(time_with_zero_tps) * time_step / 1_000_000  # Convert microseconds to seconds
total_time_zero_tps_hours = total_time_zero_tps / 3600  # Convert seconds to hours

print(f"Total time spent with 0 TPS: {total_time_zero_tps:.2f} seconds ({total_time_zero_tps_hours:.2f} hours)")

In [ ]:
# Display every 10 seconds
filtered_time_series = time_series[time_series['t'] % 10000000 == 0]
# from micro sec to sec
filtered_time_series['t'] = np.floor(filtered_time_series['t'] / 1000000)

filtered_time_series

In [ ]:
# Calculate time elapsed in seconds from the start of the log
start_time = filtered_time_series['t'].min()  # First timestamp
filtered_time_series['elapsed_time'] = filtered_time_series['t'] - start_time
filtered_time_series['elapsed_time_hours'] = filtered_time_series['elapsed_time'] / 3600

# Convert elapsed time to HH:MM:SS format
filtered_time_series['elapsed_time_formatted'] = filtered_time_series['elapsed_time'].apply(
    lambda x: f"{int(x // 3600):02}:{int((x % 3600) // 60):02}:{int(x % 60):02}"
)

In [ ]:
# Determine the range for the x-axis
x_min = filtered_time_series['elapsed_time_hours'].min()
x_max = filtered_time_series['elapsed_time_hours'].max()

fig = go.Figure()

# fig.add_trace(go.Scatter(x=filtered_time_series['elapsed_time_hours'], y=filtered_time_series['sum_tps'], mode='markers', name='TPS', marker=dict(size=2)))
fig.add_trace(go.Scatter(x=filtered_time_series['elapsed_time_hours'], y=filtered_time_series['moving_average'], mode='lines', name='TPS Moving Average(1sec)', line=dict(width=0.5)))

# Add horizontal lines for max and average TPS
fig.add_hline(y=average_tps, line_dash="dot", line_color="green", annotation=None)
fig.add_hline(y=max_tps, line_dash="dash", line_color="red", annotation_text=f"Max TPS: {max_tps:.2f}", annotation_position="top right")

fig.update_layout(
    showlegend=False,
    title=None,
    font=dict(
        family="serif",
        size=20
    ),
    xaxis=dict(
        title=None,
        tickmode='linear',
        dtick=1,
        tickformat=".1f",
        range=[x_min, x_max],
        anchor="y",
        position=0
    ),
    yaxis=dict(
        title=None,
        range=[0, 15000],
        anchor="x",
        position=0,
        tickformat="~s",
    ),
    margin=dict(l=20, r=20, t=20, b=20),
    template="simple_white",
    width=800,
    height=400
)

fig.write_image("128GB_Exec_TPS_0.pdf")
fig.show()
